In [1]:
import os

import numpy as np
import h5py
import pandas as pd

In [2]:
def calc_dice(X, Y):
    if X.any() or Y.any():
        d = 2* np.count_nonzero(np.logical_and(X, Y)) / (np.count_nonzero(X)+np.count_nonzero(Y))
    else:
        d=0
    return d


In [5]:
for idx_start in [10,55]:
    files = [h5py.File(os.path.join('D:\BiGART_results', '2d__bigart_test_{}'.format(idx_start+i),
                'test', 'prediction_test.h5' ), 'r') for i in range(5)]

    res_d1 = []

    p_comb = []
    x_comb = []
    y_comb = []
    
    
    
    
    for patient in files[0]['predicted'].keys():
        predictions = 0.2*(files[0]['predicted'][patient][:,:,:,:] \
                      + files[1]['predicted'][patient][:,:,:,:]\
                      + files[2]['predicted'][patient][:,:,:,:]\
                      + files[3]['predicted'][patient][:,:,:,:]\
                      + files[4]['predicted'][patient][:,:,:,:]) 

        p_comb.append(predictions*1) 
        x_comb.append(files[0]['x'][patient][:,:,:,:] *1) #same for all files
        y_comb.append(files[0]['y'][patient][:,:,:,:] *1) #same for all files


        predictions = predictions>0.5 
        
        gt_1 = files[0]['y'][patient][:,:,:,:]

        res_d1.append([patient, 
                       calc_dice(predictions.flatten(), gt_1.flatten())])
        
        
    
    new_file = os.path.join(r'D:\BiGART_results', 'test_voted_bigart_{}.h5'.format(idx_start))
    with h5py.File(new_file, 'w') as f:
        f.create_dataset('predicted', data=np.concatenate(p_comb))
        f.create_dataset('x', data= np.concatenate(x_comb))
        f.create_dataset('y', data= np.concatenate(y_comb))
        

    pd.DataFrame(res_d1, columns=['patient_ids', 'f1_score']).to_csv(
                    os.path.join('D:BiGART_results','bigart_{}_prediction_test_voted.csv'.format(idx_start)))
    
    print('created:', new_file)

created: D:\BiGART_results\test_voted_bigart_10.h5
created: D:\BiGART_results\test_voted_bigart_55.h5
